In [12]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
from shapely.ops import unary_union

DATA = Path("../data")
BOUNDARY_SHP = DATA / "boundaries" / "tl_2023_us_state.shp"

states = gpd.read_file(BOUNDARY_SHP)
va_raw = states.loc[states["STUSPS"] == "VA"].copy()

print("VA rows:", len(va_raw))
print("Boundary CRS:", va_raw.crs)
print("Boundary valid?:", va_raw.is_valid.iloc[0])


VA rows: 1
Boundary CRS: EPSG:4269
Boundary valid?: True


In [13]:
# Repair invalid geometry using buffer(0) on each geometry (classic GEOS fix)
va_raw["geometry"] = va_raw["geometry"].buffer(0)

print("Boundary valid after buffer(0)?:", va_raw.is_valid.iloc[0])

# Dissolve/union to a single geometry
va_geom_native = unary_union(va_raw.geometry)

# Now reproject the repaired geometry to EPSG:5070
va_geom_5070 = gpd.GeoSeries([va_geom_native], crs=va_raw.crs).to_crs("EPSG:5070").iloc[0]

# Small buffer (meters) to handle precision edge cases
va_geom_5070_buf = va_geom_5070.buffer(2000)


Boundary valid after buffer(0)?: True


In [14]:
# Fallback repair: simplify slightly (in native CRS) then buffer(0)
va_raw["geometry"] = va_raw["geometry"].simplify(0.0001).buffer(0)


In [15]:
df = pd.read_csv(DATA / "flight_cleaned_va_all_years.csv")
df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
).to_crs("EPSG:5070")

valid_mask = gdf.geometry.apply(va_geom_5070_buf.covers)

valid_points = gdf[valid_mask].copy()
outside_points = gdf[~valid_mask].copy()

print("Valid points:", len(valid_points))
print("Outside VA:", len(outside_points))


Valid points: 6
Outside VA: 2050


In [9]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path

DATA = Path("../data")
CURATED = DATA / "curated"
REJECTS = CURATED / "rejects"
REJECTS.mkdir(parents=True, exist_ok=True)

# --- Load points (already clean) ---
df = pd.read_csv(DATA / "flight_cleaned_va_all_years.csv")
df["latitude"]  = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
).to_crs("EPSG:5070")

# --- Load boundary ---
states = gpd.read_file(DATA / "boundaries" / "tl_2023_us_state.shp")

# Sanity: confirm we really have VA rows
print("Has STUSPS?", "STUSPS" in states.columns)
print("Unique STUSPS count:", states["STUSPS"].nunique())

va = states.loc[states["STUSPS"] == "VA"].to_crs("EPSG:5070")
print("VA rows in shapefile:", len(va))

# --- Dissolve/union into ONE geometry ---
va_geom = va.geometry.unary_union

# Repair geometry (two safe options)
# Option 1: Shapely 2.x
try:
    from shapely.make_valid import make_valid
    va_geom = make_valid(va_geom)
except Exception:
    # Option 2: classic "buffer(0)" repair
    va_geom = va_geom.buffer(0)

# Tiny buffer to avoid precision / coastline weirdness (meters, EPSG:5070)
va_geom_buf = va_geom.buffer(2000)

# --- Validate points ---
valid_mask = gdf.intersects(va_geom_buf)

valid_points = gdf.loc[valid_mask].copy()
outside_points = gdf.loc[~valid_mask].copy()

print("Valid points:", len(valid_points))
print("Outside VA:", len(outside_points))

valid_points.to_csv(CURATED / "facilities_va_validated.csv", index=False)
outside_points.to_csv(REJECTS / "rejects_outside_va_polygon.csv", index=False)


Has STUSPS? True
Unique STUSPS count: 56
VA rows in shapefile: 1


C:\Users\blake\AppData\Local\Temp\ipykernel_11752\1833943688.py:33: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  va_geom = va.geometry.unary_union


Valid points: 6
Outside VA: 2050


In [10]:
try:
    va_geom_ll = va.geometry.union_all()   # GeoPandas newer
except Exception:
    va_geom_ll = va.geometry.unary_union   # fallback

print("VA geom type (lon/lat CRS):", va_geom_ll.geom_type)


VA geom type (lon/lat CRS): Polygon


In [11]:
from shapely.validation import make_valid  # if available

# ensure boundary is in lon/lat first
va_ll = states.loc[states["STUSPS"] == "VA"].to_crs("EPSG:4326")

try:
    va_geom_ll = va_ll.geometry.union_all()
except Exception:
    va_geom_ll = va_ll.geometry.unary_union

# Repair in lon/lat
try:
    va_geom_ll = make_valid(va_geom_ll)
except Exception:
    va_geom_ll = va_geom_ll.buffer(0)

# Now project the repaired geometry
va_geom = gpd.GeoSeries([va_geom_ll], crs="EPSG:4326").to_crs("EPSG:5070").iloc[0]

# Slight buffer (meters)
va_geom_buf = va_geom.buffer(2000)


GEOSException: IllegalArgumentException: Points of LinearRing do not form a closed linestring

In [7]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path

DATA = Path("../data")
CURATED = DATA / "curated"
REJECTS = CURATED / "rejects"
REJECTS.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(DATA / "flight_cleaned_va_all_years.csv")

# 1) Coerce lat/lon to numeric
df["latitude"]  = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")

# 2) Keep only finite coords
finite_mask = np.isfinite(df["latitude"]) & np.isfinite(df["longitude"])
missing_coords = df[~finite_mask].copy()
df = df[finite_mask].copy()

print("Rows with missing/invalid coords:", len(missing_coords))
missing_coords.to_csv(REJECTS / "rejects_missing_or_invalid_coords.csv", index=False)

# 3) Optional: basic VA-ish range filter in EPSG:4326 (helps catch garbage coords)
range_mask = (
    df["latitude"].between(35.0, 40.5) &
    df["longitude"].between(-84.0, -74.0)
)
range_rejects = df[~range_mask].copy()
df = df[range_mask].copy()

print("Rows rejected by VA-ish lat/lon range:", len(range_rejects))
range_rejects.to_csv(REJECTS / "rejects_out_of_latlon_range.csv", index=False)

# 4) Now build points safely
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
)

# quick check: bounds should be finite
print("Point bounds (lon/lat):", gdf.total_bounds)


Rows with missing/invalid coords: 0
Rows rejected by VA-ish lat/lon range: 0
Point bounds (lon/lat): [-83.0331    36.558914 -75.54      39.25669 ]


In [8]:
va_boundary = gpd.read_file(DATA / "boundaries" / "tl_2023_us_state.shp")
va_boundary = va_boundary[va_boundary["STUSPS"] == "VA"].to_crs("EPSG:5070")

gdf_5070 = gdf.to_crs("EPSG:5070")

joined = gpd.sjoin(
    gdf_5070,
    va_boundary[["geometry"]],
    how="left",
    predicate="intersects"
)

valid_points = joined[joined.index_right.notna()].drop(columns=["index_right"])
outside_points = joined[joined.index_right.isna()].drop(columns=["index_right"])

print("Valid points:", len(valid_points))
print("Outside VA:", len(outside_points))

valid_points.to_csv(CURATED / "facilities_va_validated.csv", index=False)
outside_points.to_csv(REJECTS / "rejects_outside_va_polygon.csv", index=False)


Valid points: 6
Outside VA: 2050


In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path


In [2]:
DATA = Path("../data")

va = pd.read_csv(DATA / "flight_cleaned_va_all_years.csv")

va_boundary = gpd.read_file(
    DATA / "boundaries" / "tl_2023_us_state.shp"
)

va_boundary = va_boundary[va_boundary["STUSPS"] == "VA"]


In [3]:
gdf = gpd.GeoDataFrame(
    va,
    geometry=gpd.points_from_xy(va.longitude, va.latitude),
    crs="EPSG:4326"
)


In [4]:
gdf = gdf.to_crs("EPSG:5070")
va_boundary = va_boundary.to_crs("EPSG:5070")


In [5]:
inside_mask = gdf.within(va_boundary.iloc[0].geometry)

valid_points = gdf[inside_mask].copy()
outside_points = gdf[~inside_mask].copy()

print("Valid points:", len(valid_points))
print("Outside VA:", len(outside_points))


Valid points: 6
Outside VA: 2050


In [6]:
print("Points CRS:", gdf.crs)
print("Boundary CRS:", va_boundary.crs)

print("Points bounds:", gdf.total_bounds)         # [minx, miny, maxx, maxy]
print("Boundary bounds:", va_boundary.total_bounds)


Points CRS: EPSG:5070
Boundary CRS: EPSG:5070
Points bounds: [1463821.36848757 1695094.33562308              inf              inf]
Boundary bounds: [1089285.3344151  1576761.17536789 1796972.23964475 1966514.10100916]


In [ ]:
CURATED = DATA / "curated"
CURATED.mkdir(exist_ok=True)

valid_points.to_csv(CURATED / "facilities_va_validated.csv", index=False)
outside_points.to_csv(CURATED / "rejects_outside_va.csv", index=False)

print("Saved validated dataset")
